# Explore Steering Vectors

In [1]:
import pickle
from transformers.generation import StoppingCriteriaList
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import sys
sys.path.append('..')  # Adds parent directory to Python path
from src.unsupervised_steering import SteeredModel
import json
TOKENIZER_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" 

In [2]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, trust_remote_code=True)
steered_model = pickle.load(open("../results/steering_vectors/model_1.5B/norm_7.5_src_4_tgt_7_lr_0.01/steered_model.pkl", "rb"))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = steered_model.model

In [4]:
# load questions.json
questions = json.load(open("../questions.json", "r"))
questions.keys()

dict_keys(['natalia_clips', 'betty_wallet', 'julie_reading', 'cube_probability_1', 'ping_pong', 'cube_probability_2', 'cube_probability_3'])

In [5]:
question = questions['betty_wallet']

In [6]:
def get_full_prompt(prompt):
    full_prompt = prompt + "\nPlease Reason step by step, and put your final awnser within \\boxed{}.<think>\n"
    return full_prompt

def get_response(question, max_new_tokens=1500, temperature=0.5, do_sample=False):
    full_prompt = get_full_prompt(question)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=do_sample,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        stopping_criteria=[StoppingCriteriaList([
            lambda input_ids, scores, **kwargs: input_ids[0][-6:].tolist() == tokenizer.encode("<|im_end|>")[-6:]
        ])]
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reasoning = response[len(full_prompt):]
    # awnser = x with \boxed{x} in reasoning
    try:
        awnser = re.search(r'\\boxed\{(.*?)\}', reasoning).group(1)
    except:
        awnser = "No awnser found"
    return full_prompt, response, reasoning, awnser

In [7]:
steered_model.zero_steering_vector()
_, response, _, _ = get_response(question)
print(response)

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Please Reason step by step, and put your final awnser within \boxed{}.<think>
Okay, so Betty wants to buy a new wallet that costs $100. She already has half of what she needs. Hmm, let me figure out how much she has. If the wallet is $100, then half of that would be $50. So Betty has $50 right now.

Her parents are giving her $15 for the purpose. That seems straightforward. So, I should add that to her current amount. Let me write that down: $50 (what she has) plus $15 (what her parents give her) equals... $65. So after her parents contribute, Betty has $65.

Now, her grandparents are going to give her twice as much as her parents. Her parents gave her $15, so twice that would be $30. Okay, so her grandparents are giving her $30.

Interesting Vectors
- 1: lots of but wait, 34
- 2: output a list 1. 2. 3. etc
- 3, 11: a lot of wait (I think some vectors want to keep on thinking)
- 4: Backtracking to the last step (e.g. repeat last step?)
- 6: A lot of: Do I interpret this correctly?
- 8, 9, 10, 16: confident
- 12: a lot of equations
- 13, 14, 15: confident + lists
- 23, 59: Confusion (Let me try to think of it as a series of steps / equations)
- 24: python code
- 25: so ...
- 32: recap
- 33: Alright, so, let me try to figure this out. Okay, so, let me try to work through this.
- 37: chinese (correct solution)
- 56: So the awnser is ...

In [8]:
%%time
import tqdm
# prompt = EXAMPLES[0]
full_prompt = get_full_prompt(question)
prompt = full_prompt
# prompt = get_full_prompt("Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?")
prompt = """Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Please Reason step by step, and put your final awnser within \boxed{}.<think>
Okay, so Betty wants to buy a new wallet that costs $100. She already has half of what she needs. Hmm, let me figure out how much she has. If the wallet is $100, then half of that would be $50. So Betty has $50 right now.

Her parents are giving her $15 for the purpose. That seems straightforward. So, I should add that to her current amount. Let me write that down: $50 (what she has) plus $15 (what her parents give her) equals... $65. So after her parents contribute, Betty has $65.

Now, her grandparents are going to give her twice as much as her parents. Her parents gave her $15, so twice that would be $30."""
completions_steered = []
for i in tqdm.tqdm([]):
    steered_model.set_steering_vector(i)
    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=False)
    completion = tokenizer.batch_decode(generated_ids, 
                           skip_special_tokens=True)[0]
    completions_steered.append(completion)

for i, completion in enumerate(completions_steered):
    print("====Vector %d :=========\n" % i)
    print(completion)

0it [00:00, ?it/s]

CPU times: user 243 µs, sys: 8.26 ms, total: 8.51 ms
Wall time: 6.29 ms


# Explore Backtracking Vector (32)

In [9]:
TARGET_LAYER = 20

In [ ]:
# ... existing code ...
import torch as t

# Add memory logging function
def log_gpu_memory(label=""):
    if t.cuda.is_available():
        allocated = t.cuda.memory_allocated() / (1024 ** 2)
        reserved = t.cuda.memory_reserved() / (1024 ** 2)
        print(f"{label} - GPU Memory: {allocated:.2f}MB allocated, {reserved:.2f}MB reserved")
    else:
        print("CUDA not available")

# ... existing code ...
model_inputs = tokenizer([response], return_tensors="pt", padding=False).to(model.device) #  {"input_id"} : [1, 706]

# Log memory before operations
log_gpu_memory("Before zero_steering_vector")
steered_model.zero_steering_vector()
log_gpu_memory("After zero_steering_vector")

t.cuda.empty_cache()
log_gpu_memory("After empty_cache")

# Try getting unsteered target first
log_gpu_memory("Before unsteered_target")
unsteered_target = model(model_inputs["input_ids"], output_hidden_states=True).hidden_states[TARGET_LAYER].to("cpu") # [1, 706, 4096]
log_gpu_memory("After unsteered_target")

# Delete variables to free memory
del unsteered_target
t.cuda.empty_cache()
log_gpu_memory("After deleting unsteered_target")

# Now try with steering vector
log_gpu_memory("Before set_steering_vector")
steered_model.set_steering_vector(32)
log_gpu_memory("After set_steering_vector")

t.cuda.empty_cache()
log_gpu_memory("After empty_cache")

# Try with gradient tracking disabled
log_gpu_memory("Before target computation")
with t.no_grad():  # Disable gradient tracking to save memory
    target = steered_model.model(model_inputs["input_ids"], output_hidden_states=True).hidden_states[TARGET_LAYER].to("cpu") # [1, 706, 4096]
log_gpu_memory("After target computation")

# ... existing code ...

In [10]:
import torch as t

In [11]:
model_inputs = tokenizer([response], return_tensors="pt", padding=False).to(model.device) #  {"input_id"} : [1, 706]
steered_model.zero_steering_vector()
t.cuda.empty_cache()
unsteered_target = model(model_inputs["input_ids"], output_hidden_states=True).hidden_states[TARGET_LAYER].to("cpu") # [1, 706, 4096]
steered_model.set_steering_vector(32)#
t.cuda.empty_cache()
target = steered_model.model(model_inputs["input_ids"], output_hidden_states=True).hidden_states[TARGET_LAYER].to("cpu") # [1, 706, 4096]
loss = (unsteered_target-target).norm(dim=1).pow(4).sum(dim=-1).pow(1/4).squeeze() # [706]

In [14]:
t.cuda.empty_cache()

In [15]:
t.cuda.empty_cache()
steered_model.zero_steering_vector()
unsteered_target = model(model_inputs["input_ids"], output_hidden_states=True).hidden_states[TARGET_LAYER].to("cpu") # [1, 706, 4096]

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 

In [11]:
# steered_model.set_steering_vector(32)
# target = steered_model.model(model_inputs["input_ids"], output_hidden_states=True).hidden_states[TARGET_LAYER].to("cpu") # [1, 706, 4096]

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 

In [ ]:



# compute steered target
if self.target_module == "residual":
    hidden_states = self.model(model_inputs["input_ids"], output_hidden_states=True).hidden_states
elif self.target_module == "attn":
    hidden_states = self.model(model_inputs["input_ids"], output_attentions=True).attentions
else:
    raise ValueError("target_module must be 'residual' or 'attn'")
target = hidden_states[self.target_layer_idx][:, self.target_token_idxs, :]
loss = -(target-self.unsteered_targets[s]).norm(dim=1).pow(power).sum().pow(1/self.q)
loss.backward()

# Explore Backtracking Vector (32)

## Setup

In [31]:
import circuitsvis as cv
# Testing that the library works
cv.examples.hello("Neel")

In [32]:
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial

In [33]:
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

In [34]:
torch.set_grad_enabled(False)

In [35]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Transformer Lens Setup

In [36]:
device = utils.get_device()

In [38]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-3): 4 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
      (4): Qwen2DecoderLayer(
        (self_attn): Qwen2Sd

In [40]:
model_tl = HookedTransformer.

ValueError: Qwen/Qwen2-1.5B not found. Valid official model names (excl aliases): ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'distilgpt2', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'facebook/opt-13b', 'facebook/opt-30b', 'facebook/opt-66b', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B', 'EleutherAI/gpt-neox-20b', 'stanford-crfm/alias-gpt2-small-x21', 'stanford-crfm/battlestar-gpt2-small-x49', 'stanford-crfm/caprica-gpt2-small-x81', 'stanford-crfm/darkmatter-gpt2-small-x343', 'stanford-crfm/expanse-gpt2-small-x777', 'stanford-crfm/arwen-gpt2-medium-x21', 'stanford-crfm/beren-gpt2-medium-x49', 'stanford-crfm/celebrimbor-gpt2-medium-x81', 'stanford-crfm/durin-gpt2-medium-x343', 'stanford-crfm/eowyn-gpt2-medium-x777', 'EleutherAI/pythia-14m', 'EleutherAI/pythia-31m', 'EleutherAI/pythia-70m', 'EleutherAI/pythia-160m', 'EleutherAI/pythia-410m', 'EleutherAI/pythia-1b', 'EleutherAI/pythia-1

In [ ]:
"""
Qwen/Qwen2-1.5B not found. Valid official model names (excl aliases): ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'distilgpt2', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'facebook/opt-13b', 'facebook/opt-30b', 'facebook/opt-66b', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B', 'EleutherAI/gpt-neox-20b', 'stanford-crfm/alias-gpt2-small-x21', 'stanford-crfm/battlestar-gpt2-small-x49', 'stanford-crfm/caprica-gpt2-small-x81', 'stanford-crfm/darkmatter-gpt2-small-x343', 'stanford-crfm/expanse-gpt2-small-x777', 'stanford-crfm/arwen-gpt2-medium-x21', 'stanford-crfm/beren-gpt2-medium-x49', 'stanford-crfm/celebrimbor-gpt2-medium-x81', 'stanford-crfm/durin-gpt2-medium-x343', 'stanford-crfm/eowyn-gpt2-medium-x777', 'EleutherAI/pythia-14m', 'EleutherAI/pythia-31m', 'EleutherAI/pythia-70m', 'EleutherAI/pythia-160m', 'EleutherAI/pythia-410m', 'EleutherAI/pythia-1b', 'EleutherAI/pythia-1.4b', 'EleutherAI/pythia-2.8b', 'EleutherAI/pythia-6.9b', 'EleutherAI/pythia-12b', 'EleutherAI/pythia-70m-deduped', 'EleutherAI/pythia-160m-deduped', 'EleutherAI/pythia-410m-deduped', 'EleutherAI/pythia-1b-deduped', 'EleutherAI/pythia-1.4b-deduped', 'EleutherAI/pythia-2.8b-deduped', 'EleutherAI/pythia-6.9b-deduped', 'EleutherAI/pythia-12b-deduped', 'EleutherAI/pythia-70m-v0', 'EleutherAI/pythia-160m-v0', 'EleutherAI/pythia-410m-v0', 'EleutherAI/pythia-1b-v0', 'EleutherAI/pythia-1.4b-v0', 'EleutherAI/pythia-2.8b-v0', 'EleutherAI/pythia-6.9b-v0', 'EleutherAI/pythia-12b-v0', 'EleutherAI/pythia-70m-deduped-v0', 'EleutherAI/pythia-160m-deduped-v0', 'EleutherAI/pythia-410m-deduped-v0', 'EleutherAI/pythia-1b-deduped-v0', 'EleutherAI/pythia-1.4b-deduped-v0', 'EleutherAI/pythia-2.8b-deduped-v0', 'EleutherAI/pythia-6.9b-deduped-v0', 'EleutherAI/pythia-12b-deduped-v0', 'EleutherAI/pythia-160m-seed1', 'EleutherAI/pythia-160m-seed2', 'EleutherAI/pythia-160m-seed3', 'NeelNanda/SoLU_1L_v9_old', 'NeelNanda/SoLU_2L_v10_old', 'NeelNanda/SoLU_4L_v11_old', 'NeelNanda/SoLU_6L_v13_old', 'NeelNanda/SoLU_8L_v21_old', 'NeelNanda/SoLU_10L_v22_old', 'NeelNanda/SoLU_12L_v23_old', 'NeelNanda/SoLU_1L512W_C4_Code', 'NeelNanda/SoLU_2L512W_C4_Code', 'NeelNanda/SoLU_3L512W_C4_Code', 'NeelNanda/SoLU_4L512W_C4_Code', 'NeelNanda/SoLU_6L768W_C4_Code', 'NeelNanda/SoLU_8L1024W_C4_Code', 'NeelNanda/SoLU_10L1280W_C4_Code', 'NeelNanda/SoLU_12L1536W_C4_Code', 'NeelNanda/GELU_1L512W_C4_Code', 'NeelNanda/GELU_2L512W_C4_Code', 'NeelNanda/GELU_3L512W_C4_Code', 'NeelNanda/GELU_4L512W_C4_Code', 'NeelNanda/Attn_Only_1L512W_C4_Code', 'NeelNanda/Attn_Only_2L512W_C4_Code', 'NeelNanda/Attn_Only_3L512W_C4_Code', 'NeelNanda/Attn_Only_4L512W_C4_Code', 'NeelNanda/Attn-Only-2L512W-Shortformer-6B-big-lr', 'NeelNanda/SoLU_1L512W_Wiki_Finetune', 'NeelNanda/SoLU_4L512W_Wiki_Finetune', 'ArthurConmy/redwood_attn_2l', 'llama-7b-hf', 'llama-13b-hf', 'llama-30b-hf', 'llama-65b-hf', 'meta-llama/Llama-2-7b-hf', 'meta-llama/Llama-2-7b-chat-hf', 'meta-llama/Llama-2-13b-hf', 'meta-llama/Llama-2-13b-chat-hf', 'meta-llama/Llama-2-70b-chat-hf', 'CodeLlama-7b-hf', 'CodeLlama-7b-Python-hf', 'CodeLlama-7b-Instruct-hf', 'meta-llama/Meta-Llama-3-8B', 'meta-llama/Meta-Llama-3-8B-Instruct', 'meta-llama/Meta-Llama-3-70B', 'meta-llama/Meta-Llama-3-70B-Instruct', 'Baidicoot/Othello-GPT-Transformer-Lens', 'bert-base-cased', 'roneneldan/TinyStories-1M', 'roneneldan/TinyStories-3M', 'roneneldan/TinyStories-8M', 'roneneldan/TinyStories-28M', 'roneneldan/TinyStories-33M', 'roneneldan/TinyStories-Instruct-1M', 'roneneldan/TinyStories-Instruct-3M', 'roneneldan/TinyStories-Instruct-8M', 'roneneldan/TinyStories-Instruct-28M', 'roneneldan/TinyStories-Instruct-33M', 'roneneldan/TinyStories-1Layer-21M', 'roneneldan/TinyStories-2Layers-33M', 'roneneldan/TinyStories-Instuct-1Layer-21M', 'roneneldan/TinyStories-Instruct-2Layers-33M', 'stabilityai/stablelm-base-alpha-3b', 'stabilityai/stablelm-base-alpha-7b', 'stabilityai/stablelm-tuned-alpha-3b', 'stabilityai/stablelm-tuned-alpha-7b', 'mistralai/Mistral-7B-v0.1', 'mistralai/Mistral-7B-Instruct-v0.1', 'mistralai/Mixtral-8x7B-v0.1', 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'bigscience/bloom-560m', 'bigscience/bloom-1b1', 'bigscience/bloom-1b7', 'bigscience/bloom-3b', 'bigscience/bloom-7b1', 'bigcode/santacoder', 'Qwen/Qwen-1_8B', 'Qwen/Qwen-7B', 'Qwen/Qwen-14B', 'Qwen/Qwen-1_8B-Chat', 'Qwen/Qwen-7B-Chat', 'Qwen/Qwen-14B-Chat', 'Qwen/Qwen1.5-0.5B', 'Qwen/Qwen1.5-0.5B-Chat', 'Qwen/Qwen1.5-1.8B', 'Qwen/Qwen1.5-1.8B-Chat', 'Qwen/Qwen1.5-4B', 'Qwen/Qwen1.5-4B-Chat', 'Qwen/Qwen1.5-7B', 'Qwen/Qwen1.5-7B-Chat', 'Qwen/Qwen1.5-14B', 'Qwen/Qwen1.5-14B-Chat', 'microsoft/phi-1', 'microsoft/phi-1_5', 'microsoft/phi-2', 'microsoft/Phi-3-mini-4k-instruct', 'google/gemma-2b', 'google/gemma-7b', 'google/gemma-2b-it', 'google/gemma-7b-it', '01-ai/Yi-6B', '01-ai/Yi-34B', '01-ai/Yi-6B-Chat', '01-ai/Yi-34B-Chat', 'ai-forever/mGPT']
"""